In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 40
from copy import deepcopy

In [2]:
!ls "raw data"

ACC HS Chronic Absenteeism.xlsx
ACC HS Graduation Rate.xlsx
Annual Regents Exams.xlsx
GRAD_RATE_AND_OUTCOMES_2022.csv
nys-ann_regents-reduc.csv


### Reading in Datasets

In [4]:
nys_absenteeism_df = pd.read_excel("raw data/ACC HS Chronic Absenteeism.xlsx",index_col=False, 
                                   dtype=({'ENTITY_CD':'string'}))

In [3]:
nys_annual_regents = pd.read_csv("raw data/nys-ann_regents-reduc.csv",index_col=False,
                                dtype=({'ENTITY_CD':'string','INSTITUTION_ID':'string'}))

In [6]:
nys_grad_rate_df = pd.read_excel("raw data/ACC HS Graduation Rate.xlsx",index_col=False,dtype=({'ENTITY_CD':'string'}))

### Filtering & Cleaning Datasets

In [8]:
# looking at the columns in nys_annual_regents csv 
nys_annual_regents.columns

Index(['institution_id', 'entity_cd', 'entity_name', 'year', 'subject',
       'subgroup_name', 'tested', 'num_level1', 'per_level1', 'num_level2',
       'per_level2', 'num_level3', 'per_level3', 'num_level4', 'per_level4',
       'num_level5', 'per_level5', 'num_prof', 'per_prof', 'total_exempt',
       'num_exempt_ntest', 'pct_exempt_ntest', 'num_exempt_test',
       'pct_exempt_test', 'assmnt_flag'],
      dtype='object')

In [9]:
# filtering our dataframe so that we only have records from the 2021-2022 AY
# also not including 'All Students' as an aggregate measure
nys_annual_regents = nys_annual_regents[(nys_annual_regents.year == 2022) 
                                        & (nys_annual_regents.subgroup_name != 'All Students')]

In [16]:
# for simplicity, we're interested in whether students in a given subgroup were proficient in a given subject 
ann_regs_cols = ['entity_cd','entity_name','year','subject','subgroup_name',
                 'tested','num_prof']
nys_annual_regents = nys_annual_regents.loc[:,ann_regs_cols]

In [17]:
# we also want to exclude records that might have 's' in 'num_prof' as these indicate records
# containing less than 5 students per subgroup 
nys_annual_regents = nys_annual_regents[nys_annual_regents.num_prof.str.contains('s')==False]

,entity_cd,entity_name,year,subject,subgroup_name,tested,num_prof
234654,10100010030,William S Hackett Middle School,2022,Regents Common Core Algebra I,Asian or Native Hawaiian/Other Pacific Islander,17,14
234655,10100010030,William S Hackett Middle School,2022,Regents Living Environment,Asian or Native Hawaiian/Other Pacific Islander,14,14
234658,10100010030,William S Hackett Middle School,2022,Regents Common Core Algebra I,Economically Disadvantaged,23,20
234659,10100010030,William S Hackett Middle School,2022,Regents Living Environment,Economically Disadvantaged,25,21
234661,10100010030,William S Hackett Middle School,2022,Regents Common Core Algebra I,Female,33,32
...,...,...,...,...,...,...,...
482785,680801040001,Dundee Junior-Senior High School,2022,Regents NF Global History,Students with Disabilities,8,3
482786,680801040001,Dundee Junior-Senior High School,2022,Regents Common Core Algebra II,White,16,13
482787,680801040001,Dundee Junior-Senior High School,2022,Regents US History&Gov't (Framework),White,0,0
482790,680801040001,Dundee Junior-Senior High School,2022,Regents Living Environment,White,37,32


Now we must do that with our `grad-rate` dataset

In [19]:
nys_grad_rate_df.head()

,INSTITUTION_ID,ENTITY_CD,ENTITY_NAME,YEAR,SUBGROUP_NAME,COHORT,COHORT_COUNT,GRAD_COUNT,GRAD_RATE,COHORT_LEVEL,OVERRIDE,WT_PERF_FLAG
0,800000055729,010100010000,ALBANY CITY SD,2021,American Indian or Alaska Native,5-Year,2,s,s,NaN,NaN,NaN
1,800000055729,010100010000,ALBANY CITY SD,2021,All Students,5-Year,697,517,74.2,NaN,NaN,NaN
2,800000055729,010100010000,ALBANY CITY SD,2021,Black or African American,6-Year,351,244,69.5,NaN,NaN,NaN
3,800000055729,010100010000,ALBANY CITY SD,2022,Economically Disadvantaged,4-Year,434,333,76.7,s,NaN,N
4,800000055729,010100010000,ALBANY CITY SD,2022,Hispanic or Latino,5-Year,108,78,72.2,s,NaN,N


Very messy. We only need a subset of columns. For this, we'll leverage the documentation provided to us by the database.

In [104]:
# transforming our grad rate dataset 

# normalizing column names 
nys_grad_rate_df.columns = nys_grad_rate_df.columns.str.lower()

# getting only columns of interest 
nys_grad_rate_df = nys_grad_rate_df.loc[:,['entity_cd','entity_name','year','subgroup_name','cohort','grad_rate']]

# removing all records that contain district-level aggregations 
nys_grad_rate_df = nys_grad_rate_df[~nys_grad_rate_df['entity_cd'].astype('str').str.endswith('0000')]

# removing all records containing aggregations on all public schools
nys_grad_rate_df = nys_grad_rate_df[~nys_grad_rate_df['entity_cd'].astype('str').str.endswith('111111111111')]


# removing all records containing combined aggregate measures for all students 
nys_grad_rate_df = nys_grad_rate_df[nys_grad_rate_df.subgroup_name != 'All Students']

# normalizing entity_name 
nys_grad_rate_df.entity_name = nys_grad_rate_df.entity_name.str.title()

# reseting our index 
nys_grad_rate_df = nys_grad_rate_df.reset_index(drop=True)

In [105]:
# we now have a new dataframe containing a lot of information that we still need to further reduce 
nys_grad_rate_df.head()

,entity_cd,entity_name,year,subgroup_name,cohort,grad_rate
0,010100010034,Albany High School,2022,Hispanic or Latino,6-Year,75.9
1,010100010034,Albany High School,2022,English Language Learner,4-Year,78.7
2,010100010034,Albany High School,2022,Multiracial,6-Year,s
3,010100010034,Albany High School,2022,Hispanic or Latino,Combined,73.7
4,010100010034,Albany High School,2022,White,4-Year,86.6


In [106]:
# creating a new dataframe 
# this contains graduation_rate records for the Combined cohorts in the year 2022
# also want to drop records where grad_rate is == 's' as these are restricted for privacy reasons
nys_grad_rate_22 = nys_grad_rate_df[(nys_grad_rate_df.cohort == 'Combined') & (nys_grad_rate_df.year == 2022) 
                 & (nys_grad_rate_df.grad_rate != 's')]
nys_grad_rate_22 = nys_grad_rate_22.drop(columns=['cohort'])
nys_grad_rate_22 = nys_grad_rate_22.reset_index(drop=True)

In [107]:
# we now have a dataset containing information we want 
nys_grad_rate_22.head()

,entity_cd,entity_name,year,subgroup_name,grad_rate
0,010100010034,Albany High School,2022,Hispanic or Latino,73.7
1,010100010034,Albany High School,2022,White,86.9
2,010100010034,Albany High School,2022,Multiracial,77.4
3,010100010034,Albany High School,2022,English Language Learner,70.4
4,010100010034,Albany High School,2022,Economically Disadvantaged,75.5


Final dataframe. Need to transform dataset containing information on chronic absenteeism so that we isolate records containing information on the 2021-2022 AY and are not aggregate measures for all students 

In [26]:
# our original dataset 
nys_absenteeism_df.head()

,INSTITUTION_ID,ENTITY_CD,ENTITY_NAME,YEAR,SUBJECT,SUBGROUP_NAME,ENROLLMENT,ABSENT_COUNT,ABSENT_RATE,LEVEL,OVERRIDE,DATA_REP_FLAG,PARTIAL_DATA_FLAG
0,800000055729,010100010000,ALBANY CITY SD,2021,HS_CA,All Students,2755,1031,37.4,NaN,NaN,Y,NaN
1,800000055729,010100010000,ALBANY CITY SD,2021,HS_CA,Students with Disabilities,415,197,47.5,NaN,NaN,Y,NaN
2,800000055729,010100010000,ALBANY CITY SD,2021,HS_CA,American Indian or Alaska Native,9,s,s,NaN,NaN,Y,NaN
3,800000055729,010100010000,ALBANY CITY SD,2021,HS_CA,Asian or Native Hawaiian/Other Pacific Islander,219,40,18.3,NaN,NaN,Y,NaN
4,800000055729,010100010000,ALBANY CITY SD,2021,HS_CA,Black or African American,1398,624,44.6,NaN,NaN,Y,NaN


In [38]:
# transforming our chronic absenteeism dataset

# normalizing column names 
nys_absenteeism_df.columns = nys_absenteeism_df.columns.str.lower()

# selecting columns of importance
nys_absenteeism_df = nys_absenteeism_df.loc[:,['entity_cd', 'entity_name','year','subgroup_name','absent_rate']]

# removing records that contain district-level aggregations 
nys_absenteeism_df = nys_absenteeism_df[~nys_absenteeism_df['entity_cd'].astype('str').str.endswith('0000')]

# removing records that are aggregations between all subgroups of students 
nys_absenteeism_df = nys_absenteeism_df[nys_absenteeism_df.subgroup_name != 'All Students']

# normalizing entity_name 
nys_absenteeism_df.entity_name = nys_absenteeism_df.entity_name.str.title()
nys_absenteeism_df = nys_absenteeism_df.reset_index(drop=True)
nys_absenteeism_df.head()

,entity_cd,entity_name,year,subgroup_name,absent_rate
0,010100010034,Albany High School,2021,Students with Disabilities,47.3
1,010100010034,Albany High School,2021,American Indian or Alaska Native,s
2,010100010034,Albany High School,2021,Asian or Native Hawaiian/Other Pacific Islander,18.3
3,010100010034,Albany High School,2021,Black or African American,44.6
4,010100010034,Albany High School,2021,Hispanic or Latino,38.4


## Cleaned Datasets

Our new datasets, after filtering and cleaning up a bit include: `nys_absenteeism_df`, `nys_grad_rate_22`, `nys_annual_regents`.

In [43]:
len(nys_annual_regents.entity_name)

234125

In [42]:
len(nys_absenteeism_df.entity_name)

25091

In [108]:
len(nys_grad_rate_22.entity_name)

4680

In [111]:
nys_grad_rate_22.tail()

,entity_cd,entity_name,year,subgroup_name,grad_rate
4675,671501040002,Warsaw Senior High School,2022,Economically Disadvantaged,88.1
4676,680601060001,Penn Yan Academy,2022,White,91.1
4677,680601060001,Penn Yan Academy,2022,Economically Disadvantaged,88.6
4678,680801040001,Dundee Junior-Senior High School,2022,White,78.6
4679,680801040001,Dundee Junior-Senior High School,2022,Economically Disadvantaged,68.8


In [100]:
nys_grad_rate_22.groupby(['entity_name','subgroup_name']).last()[95:103]

entity_cd  \
entity_name        subgroup_name                                                   
Albany High School Asian or Native Hawaiian/Other Pacific Islander  010100010034   
                   Black or African American                        010100010034   
                   Economically Disadvantaged                       010100010034   
                   English Language Learner                         010100010034   
                   Hispanic or Latino                               010100010034   
                   Multiracial                                      010100010034   
                   Students with Disabilities                       010100010034   
                   White                                            010100010034   

                                                                    year  \
entity_name        subgroup_name                                           
Albany High School Asian or Native Hawaiian/Other Pacific Islander  2022   
                   Black or African American                        2022   
                   Economically Disadvantaged                       2022   
                   English Language Learner                         2022   
                   Hispanic or Latino                               2022   
                   Multiracial                                      2022   
                   Students with Disabilities                       2022   
                   White                                            2022   

                                                                   grad_rate  
entity_name        subgroup_name                                              
Albany High School Asian or Native Hawaiian/Other Pacific Islander      82.4  
                   Black or African American                            77.1  
                   Economically Disadvantaged                           75.5  
                   English Language Learner                             70.4  
                   Hispanic or Latino                                   73.7  
                   Multiracial                                          77.4  
                   Students with Disabilities                           58.1  
                   White                                                86.9

In [490]:
shared_cols

['entity_cd', 'entity_name', 'year', 'subgroup_name']

In [501]:
pd.merge(pd.merge(nys_annual_regents,nys_grad_rate_22,how='inner',on=shared_cols),nys_absenteeism_df,
        how='inner',on=shared_cols)

,entity_cd,entity_name,year,subject,subgroup_name,tested,num_prof,grad_rate,absent_rate,level
0,010100010034,Albany High School,2022,Regents US History&Gov't (Framework),Asian or Native Hawaiian/Other Pacific Islander,0,0,82.4,41.8,2
1,010100010034,Albany High School,2022,Regents Phy Set/Physics,Asian or Native Hawaiian/Other Pacific Islander,22,8,82.4,41.8,2
2,010100010034,Albany High School,2022,Regents Phy Set/Chemistry,Asian or Native Hawaiian/Other Pacific Islander,37,18,82.4,41.8,2
3,010100010034,Albany High School,2022,Regents NF Global History,Asian or Native Hawaiian/Other Pacific Islander,61,46,82.4,41.8,2
4,010100010034,Albany High School,2022,Regents Living Environment,Asian or Native Hawaiian/Other Pacific Islander,40,22,82.4,41.8,2
...,...,...,...,...,...,...,...,...,...,...
42642,680801040001,Dundee Junior-Senior High School,2022,Regents Living Environment,White,37,32,78.6,35.1,3
42643,680801040001,Dundee Junior-Senior High School,2022,Regents NF Global History,White,34,s,78.6,35.1,3
42644,680801040001,Dundee Junior-Senior High School,2022,Regents Phy Set/Chemistry,White,11,5,78.6,35.1,3
42645,680801040001,Dundee Junior-Senior High School,2022,Regents Phy Set/Earth Sci,White,27,s,78.6,35.1,3


Now, we need to merge the two columns to have a total of 4 shared columns and 5 features, thus having 9 columns in our final dataframe

In [118]:
#shared_cols = ['entity_cd','entity_name','year','subgroup_name']

In [121]:
#pd.merge(pd.merge(nys_absenteeism_df,nys_assessment_df, on = shared_cols),nys_grad_rate_df, on = shared_cols)

In [128]:
nys_final_df = pd.merge(nys_absenteeism_df,nys_grad_rate_df,how='inner',on=['entity_cd','entity_name','year','subgroup_name'])

In [130]:
nys_final_df

,entity_cd,entity_name,year,subgroup_name,enrollment,absent_count,cohort,cohort_count,grad_count
0,010100010034,ALBANY HIGH SCHOOL,2021,Students with Disabilities,414,196,6-Year,93,48
1,010100010034,ALBANY HIGH SCHOOL,2021,Students with Disabilities,414,196,5-Year,77,42
2,010100010034,ALBANY HIGH SCHOOL,2021,Students with Disabilities,414,196,4-Year,89,45
3,010100010034,ALBANY HIGH SCHOOL,2021,American Indian or Alaska Native,9,s,6-Year,3,s
4,010100010034,ALBANY HIGH SCHOOL,2021,American Indian or Alaska Native,9,s,5-Year,2,s
...,...,...,...,...,...,...,...,...,...
66280,680801040002,DUNDEE ELEMENTARY SCHOOL,2022,Economically Disadvantaged,s,s,5-Year,0,s
66281,680801040002,DUNDEE ELEMENTARY SCHOOL,2022,Economically Disadvantaged,s,s,6-Year,0,s
66282,680801040002,DUNDEE ELEMENTARY SCHOOL,2022,English Language Learner,s,s,4-Year,0,s
66283,680801040002,DUNDEE ELEMENTARY SCHOOL,2022,English Language Learner,s,s,6-Year,0,s


In [ ]:
nys_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66285 entries, 0 to 66284
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   entity_cd      66285 non-null  int64 
 1   entity_name    66285 non-null  object
 2   year           66285 non-null  int64 
 3   subgroup_name  66285 non-null  object
 4   enrollment     66285 non-null  object
 5   absent_count   66285 non-null  object
 6   cohort         66285 non-null  object
 7   cohort_count   66285 non-null  object
 8   grad_count     66285 non-null  object
dtypes: int64(2), object(7)
memory usage: 5.1+ MB


In [ ]:
nys_final_df = nys_final_df[nys_final_df.grad_count.str.contains('s')==False]
nys_final_df = nys_final_df[nys_final_df.absent_count.str.contains('s') == False]

In [ ]:
nys_final_df

,entity_cd,entity_name,year,subgroup_name,enrollment,absent_count,cohort,cohort_count,grad_count
0,10100010034,ALBANY HIGH SCHOOL,2021,Students with Disabilities,414,196,6-Year,93,48
1,10100010034,ALBANY HIGH SCHOOL,2021,Students with Disabilities,414,196,5-Year,77,42
2,10100010034,ALBANY HIGH SCHOOL,2021,Students with Disabilities,414,196,4-Year,89,45
6,10100010034,ALBANY HIGH SCHOOL,2021,Asian or Native Hawaiian/Other Pacific Islander,219,40,4-Year,57,48
7,10100010034,ALBANY HIGH SCHOOL,2021,Asian or Native Hawaiian/Other Pacific Islander,219,40,6-Year,53,44
...,...,...,...,...,...,...,...,...,...
66227,680601060001,PENN YAN ACADEMY,2022,Economically Disadvantaged,191,67,6-Year,65,58
66246,680801040001,DUNDEE JUNIOR-SENIOR HIGH SCHOOL,2022,White,174,61,6-Year,54,46
66247,680801040001,DUNDEE JUNIOR-SENIOR HIGH SCHOOL,2022,White,174,61,5-Year,40,30
66248,680801040001,DUNDEE JUNIOR-SENIOR HIGH SCHOOL,2022,White,174,61,4-Year,53,40


In [185]:
copy_1[copy_1.cohort == '6-Year']

,entity_cd,entity_name,year,subgroup_name,cohort,cohort_count,grad_count
5,010100010034,ALBANY HIGH SCHOOL,2022,White,6-Year,139,125
54,010100010034,ALBANY HIGH SCHOOL,2021,White,6-Year,129,101


In [186]:
copy_2[copy_2.membership_desc.str.contains('6')]

,aggregation_code,aggregation_name,nrc_desc,membership_desc,subgroup_name,grad_cnt
81839,010100010034,ALBANY HIGH SCHOOL,Urban-Suburban High Needs,2016 Total Cohort - 6 Year Outcome,White,122
81948,010100010034,ALBANY HIGH SCHOOL,Urban-Suburban High Needs,2016 Total Cohort - 6 Year Outcome - August 2022,White,122


In [262]:
nys_grad_rate_df

,entity_cd,entity_name,year,subgroup_name,cohort,grad_count
0,010100010034,ALBANY HIGH SCHOOL,2022,Hispanic or Latino,6-Year,82
1,010100010034,ALBANY HIGH SCHOOL,2022,English Language Learner,4-Year,74
2,010100010034,ALBANY HIGH SCHOOL,2022,Multiracial,6-Year,s
3,010100010034,ALBANY HIGH SCHOOL,2022,White,4-Year,123
4,010100010034,ALBANY HIGH SCHOOL,2022,White,5-Year,122
...,...,...,...,...,...,...
71221,680801040002,DUNDEE ELEMENTARY SCHOOL,2022,Asian or Native Hawaiian/Other Pacific Islander,6-Year,s
71222,680801040002,DUNDEE ELEMENTARY SCHOOL,2022,Black or African American,4-Year,s
71223,680801040002,DUNDEE ELEMENTARY SCHOOL,2022,Black or African American,5-Year,s
71224,680801040002,DUNDEE ELEMENTARY SCHOOL,2022,Black or African American,6-Year,s


## Exporting Cleaned Data 

In [518]:
#nys_grad_rate_22.to_csv('data/2022_NYS_grad-rate.csv',index=False)